In [ ]:
# Import libraries
from pathlib import Path
import acoular as ac
import numpy as np
import pyfar as pf
import time
initialTime = time.time()

#---------------------------------------------------------------------------------------------------------------
# initial definitions
# name of result file
resFileName = "out/2025-01-28_15-59-01_400437_eval"

# define microphone geometry
micgeofile = Path().resolve() / "in/array_position_data/bassoon_cage_64_optimized.xml"
m = ac.MicGeom(from_file=micgeofile)

# define calculation grid
resolution = 0.02
g = ac.RectGrid3D(x_min=0.35, x_max=1.15, y_min=0.15, y_max=1.15, z_min=0.25, z_max=1.65, increment = resolution)

# define audio data
name = 'in/array_audio_data/2025-01-28_15-59-01_400437.h5'
data = ac.MaskedTimeSamples(name = name)

# define time related variables
start = 0#83#26        # in s
stop = 105#35#92#36
framerate = 1           # in f/s

# define freqency resolution
bandwith = 1            # Octave = 1; 1/3 Octave = 3
frequency_bands = [1000] # Array with mid frequencies of frequency bands

#---------------------------------------------------------------------------------------------------------------
# calculation of fixed values
# st = ac.SteeringVector(grid=g, mics=m, steer_type='true location')
frameLength = 1/framerate

print('init time: ' + str(time.time()-initialTime) + ' s')

result = np.zeros([len(frequency_bands),int((stop-start)*framerate),g.nxsteps,g.nysteps,g.nzsteps])


init time: 0.05930924415588379 s


In [ ]:

# begin evaluation loop
calcTime = 0
i = 0
for i, currentFreqBand in enumerate(frequency_bands):

    frame = 0
    for frame in range(0,int((stop-start)*framerate)):
        calcTime = time.time()

        print('Frame length: ' + str((start+(frameLength*(frame+1)))-(start+(frameLength*frame))) + ' s ' + '(Frame ' + str(frame)+')')
        print('Frequency Band: ' + str(i) + ' (' + str(currentFreqBand) + ' Hz)')

        data = ac.MaskedTimeSamples(name = name)
        data = ac.MaskedTimeSamples(name = name, start = (start+(frameLength*frame))*data.sample_freq, stop = (start+(frameLength*(frame+1)))*data.sample_freq)
        # print('length of cropped measurement: ' + str(data.numsamples/data.sample_freq) + ' s (' + str(data.numsamples) + ' samples)')

        f = ac.PowerSpectra(source=data, window='Hanning', overlap='50%', block_size=512)#256)#128)
        st = ac.SteeringVector(grid=g, mics=m, steer_type='true location')
        b = ac.BeamformerCleansc(freq_data=f, steer=st)

        result[i,frame] = b.synthetic(currentFreqBand, bandwith)

        print('Frame ' + str(frame) + ' calculated. Calculation time:' + str(time.time()-calcTime))
        frame = frame + 1
    
    i = i+1

Frame length: 0.25 s (Frame 0)
Frequency Band: 0 (1000 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 5)]
[('2025-01-28_15-59-01_400437_cache.h5', 6)]
Frame 0 calculated. Calculation time:7.073791265487671
Frame length: 0.25 s (Frame 1)
Frequency Band: 0 (1000 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 7)]
[('2025-01-28_15-59-01_400437_cache.h5', 8)]
Frame 1 calculated. Calculation time:6.20820689201355
Frame length: 0.25 s (Frame 2)
Frequency Band: 0 (1000 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 9)]
[('2025-01-28_15-59-01_400437_cache.h5', 10)]
Frame 2 calculated. Calculation time:6.198883533477783
Frame length: 0.25 s (Frame 3)
Frequency Band: 0 (1000 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 11)]
[('2025-01-28_15-59-01_400437_cache.h5', 12)]
Frame 3 calculated. Calculation time:6.816716432571411
Frame length: 0.25 s (Frame 4)
Frequency Band: 0 (1000 Hz)
[('2025-01-28_15-59-01_400437_cache.h5', 13)]
[('2025-01-28_15-59-01_400437_cache.h5', 14)]
Frame 4 calculated. Calculation

In [6]:
# save Data
np.save(resFileName,result)
pf.io.write(resFileName, 
           start = start, 
           stop = stop, 
           framerate = framerate, 
           frames = int((stop-start)*framerate), 
           frequency_bands = frequency_bands, 
           bandwith = bandwith,
           resolution = resolution,
           name = name)

In [9]:
# read Data

result = np.load(resFileName + ".npy")
start, stop, framerate, frames, frequency_bands, bandwith, resolution, name = pf.io.read(resFileName+".far")
# result_Lp = ac.L_p(result)        # convert results in sound pressure Level